In [ ]:
import scanpy as sc
import numpy as np
import pandas as pd
import anndata

import random


/dccstor/bmfm-targets/users/liransz/envs/targets_env/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_csv from `anndata` is deprecated. Import anndata.io.read_csv instead.
  warnings.warn(msg, FutureWarning)
/dccstor/bmfm-targets/users/liransz/envs/targets_env/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_excel from `anndata` is deprecated. Import anndata.io.read_excel instead.
  warnings.warn(msg, FutureWarning)
/dccstor/bmfm-targets/users/liransz/envs/targets_env/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_hdf from `anndata` is deprecated. Import anndata.io.read_hdf instead.
  warnings.warn(msg, FutureWarning)
/dccstor/bmfm-targets/users/liransz/envs/targets_env/lib/python3.11/site-packages/anndata/utils.py:429: FutureWarning: Importing read_loom from `anndata` is deprecated. Import anndata.io.read_loom instead.
  warnings.warn(msg, FutureWarning)
/dccstor/bmfm-targets/users/lirans

In [32]:
adata = sc.read_h5ad("/dccstor/bmfm-targets/users/liransz/code/bmfm-targets/bmfm_targets/tests/resources/finetune/zheng68k/h5ad/zheng68k.h5ad") 
adata

AnnData object with n_obs × n_vars = 165 × 9621
    obs: 'celltype', 'cell_type_ontology_term_id'
    var: 'gene_ids', 'n_cells'

In [39]:
def create_mock_data(adata, output_filename):
    num_cells, num_genes = adata.shape
    original_cell_names = adata.obs_names

    L = ["A", "C", "G", "T"]
    new_cell_names = [
        "".join(random.choices(L, k=14)) + "-" + str(random.randint(1, 9))
        for _ in range(num_cells)
    ]
    gene_names = adata.var_names

    # Generate new gene expression values by sampling from the original distribution
    mock_X = np.zeros((num_cells, num_genes))

    for i in range(num_genes):
        gene_values = adata.X[:, i].toarray().flatten() if hasattr(adata.X, "toarray") else adata.X[:, i]
        mean, std = np.mean(gene_values), np.std(gene_values)
        mock_X[:, i] = np.random.normal(loc=mean, scale=std, size=num_cells)

    mock_X = np.clip(mock_X, 0, None)

    mock_adata = anndata.AnnData(X=mock_X, obs=pd.DataFrame(index=new_cell_names), var=pd.DataFrame(index=gene_names))

    new_obs_data = {}
    for col in adata.obs.columns:
        col_unique = adata.obs[col].unique()
        new_obs_data[col] = np.random.choice(col_unique, size=num_cells, replace=True)

    new_obs = pd.DataFrame(new_obs_data, index=mock_adata.obs_names)  

    mock_adata.obs = new_obs
    mock_adata.var = adata.var
    mock_adata.uns = adata.uns

    mock_adata.write(f"{output_filename}.h5ad")
    return mock_adata

    

In [34]:
mock_adata = create_mock_data(adata, 'zheng68k')

In [35]:
mock_adata

AnnData object with n_obs × n_vars = 165 × 9621
    obs: 'celltype', 'cell_type_ontology_term_id'

In [40]:
adata = sc.read_h5ad("/dccstor/bmfm-targets/users/liransz/code/bmfm-targets/bmfm_targets/tests/resources/finetune/zheng68k/processed/dev/processed_final.h5ad") 
adata

AnnData object with n_obs × n_vars = 15 × 4074
    obs: 'celltype', 'n_counts'
    var: 'gene_ids', 'n_cells', 'n_counts'
    uns: 'log1p'

In [41]:
mock_adata_processed = create_mock_data(adata, 'processed_final_dev')

In [44]:
adata = sc.read_h5ad("/dccstor/bmfm-targets/users/liransz/code/bmfm-targets/bmfm_targets/tests/resources/finetune/zheng68k/processed/train/processed_final.h5ad") 
adata

AnnData object with n_obs × n_vars = 133 × 9069
    obs: 'celltype', 'n_counts'
    var: 'gene_ids', 'n_cells', 'n_counts'
    uns: 'log1p'

In [45]:
mock_adata_processed = create_mock_data(adata, 'processed_final_train')